In [17]:
import requests
import json 
import os

In [18]:
#The api for tickers that the user will decide from
crypto_url = "https://api.coingecko.com/api/v3/exchanges/{0}/tickers?coin_ids={1}"
#list of markets places user can purchase crypto and cryptos available
top_five = ["gemini", "gdax", "kraken", "ftx_spot"]
crypto_list = ["ethereum", "bitcoin", "dogecoin", "matic-network"] #matic network doesnt work properly when called through index in the paramater

In [23]:
# running a loop to find each exchange that trades the coins listed in crypto_list to USD
def coin(crypto):
    for exchange in top_five:
        platform = requests.get(crypto_url.format(exchange, crypto)).json()
        for currency in platform["tickers"]:
            if currency["target"] == "USD":
                print(json.dumps(currency,indent = 4, sort_keys = True))
                #base = platform["base"]
                #last = platform["last"]
                #market = platform["market"]["name"]
                #time = str(platform["timestamp"])[0:10]

In [24]:
print(coin(crypto_list[2]))

{
    "base": "DOGE",
    "bid_ask_spread_percentage": 0.060354,
    "coin_id": "dogecoin",
    "converted_last": {
        "btc": 3.39e-06,
        "eth": 4.768e-05,
        "usd": 0.14908
    },
    "converted_volume": {
        "btc": 13.303619,
        "eth": 187.367,
        "usd": 585853
    },
    "is_anomaly": false,
    "is_stale": false,
    "last": 0.14908,
    "last_fetch_at": "2022-02-17T03:31:58+00:00",
    "last_traded_at": "2022-02-17T03:31:58+00:00",
    "market": {
        "has_trading_incentive": false,
        "identifier": "gemini",
        "name": "Gemini"
    },
    "target": "USD",
    "timestamp": "2022-02-17T03:31:58+00:00",
    "token_info_url": null,
    "trade_url": null,
    "trust_score": "green",
    "volume": 3929790.136797
}
{
    "base": "DOGE",
    "bid_ask_spread_percentage": 0.067114,
    "coin_id": "dogecoin",
    "converted_last": {
        "btc": 3.39e-06,
        "eth": 4.773e-05,
        "usd": 0.149
    },
    "converted_volume": {
        "b